### TODO:
1. Back story: how human solve problem under uncertainty
2. Hard problems 
3. Need for a real-world feasible and affordable solution
4. find quotes 

### plan
1. the story for task scheduling 
2. develop version 1 (simplified problem)
3. develop version 2 (sophisticated algorithm)
4. develop version 3 (solver based)
5. develop version 4 (combucb1 bandit)
6. More examples: shortest path (TS-comb)
7. More examples: matching (Linear Bayesian with TF-probability)


In [6]:
# !pip install tabulate

# The Dauntless Algorithms

-- TODO: Change the paragraph to the purpose of the chapter

In many real-world situations, we face a well-known problem like scheduling a set of conflicting tasks or finding the shortest path. Although we know much about these problems and have good algorithms to solve them, we often run into the frustrating case of not having full info needed to run the algorithm. For example, we may know the tasks we want to schedule but do not know who much time each task will take. This section will introduce you to algorithms that will help you run your favorite algorithm to solve problems you are not 100% certain about. To build intuition, we will use task scheduling as a running example. We will start with a simple educational version of the problem and gradually add some complexity that you may face in real-world scenarios. At the end of the chapter, we will reach a dauntless algorithm that will enable you to solve complex problems that you are not even certain about. With no further do let's get started!


### How many phone calls?
Our friend Reco got an interesting job in a marketing company, he has the number of clients he needs to contact to market the company products to them. Reco has access to the clients' calendar, and he is working on picking time slots to make the best of his time, the goal is to talk to as many clients as he can.

<img src="../figures/Constraint graph.PNG" width="600px"/>


### Version1: the activity selection problem
As usual, Reko starts with simple solutions first. Reco noticed if we have only one slot per day for each client this problem is similar to the activity scheduling problem. In this problem, the input is a set of tasks and there start and end time, the tasks have conflicting periods and one person can only perform one task at a time. The goal is to select the maximum number of tasks that can be executed by one person. One solution for activity selection problem is the greedy solution, you start with the task that finish first then pick tasks that finish first and does not conflict with the last task you have selected.

In [1]:
tasks = [("T1", 8.50, 9.50), 
         ("T2", 9.25, 10.25),
         ("T3", 9.30, 10.75),
         ("T4", 10.25, 11.50),
         ("T5", 12.00, 13.50),
         ("T6", 12.25, 13.75)]


In [2]:
# Algorithm: Activity selection problem
# inputs: tasks with start and finish times
# output: selected_tasks list of maximum none confilecting tasks

#1. stort data by finish time
tasks = sorted(tasks, key=lambda x: x[2])

#2. start with the taks that have the earliest finsh time
last_selected_task = tasks.pop(0)
last_selected_task_finish = last_selected_task[2]
selected_tasks = [last_selected_task]

# add tasks with earlist finish that does not conflict with the seleected tasks
while len(tasks) > 0:
    task = tasks.pop(0)
    task_start = task[1]
    if task_start >= last_selected_task_finish:
        last_selected_task_finish = task[2]
        selected_tasks += [task]


print (selected_tasks)

[('T1', 8.5, 9.5), ('T4', 10.25, 11.5), ('T5', 12.0, 13.5)]


In [3]:
import pandas as pd
from tabulate import tabulate
import math

df = pd.DataFrame(selected_tasks, columns=["task", "start time", "finish time"]).set_index("task")
print(tabulate(df, headers='keys', tablefmt='psql'))


ModuleNotFoundError: No module named 'tabulate'

By applying the greedy algorithm on our example, Reco can talk to 4 clients on that day, Reco can call Clients C1, C3 and C5 with no conflicts. The greedy solution is optimal, to see that, let’s assume we have an optimal solution that does not start with the first finishing task, if we replace the first task in the solution with the task that is earliest finishing task (note that no time conflict here), then we will have another optimal solution. Now we need to select the optimal tasks from the remaining tasks, so we are solving the Activity selection problem again on small problem size, we can apply the same logic until the smaller problem reach size zero. 

This solution seems to work fine, however, we could better model our problem. first, in reality, Reco will not use all the slot time to make the call, for example, if the slot is 45 Mins, Reco may only use 30 Mis of them. Moreover, clients have different concerns and personalities, for easy-going clients, Reco could finish in 15 Mins and for other less pleasant clients, he may need up to an hour. Second, some clients are more valuable to the company, so Reco needs to take client value into consideration. To handle these issues, Reko thought of another iteration of the problem, it is called the weighted activity selection problem.

### Version 2: Weighted activity selection problem
In this version, Reco divided each time slot on 15 min bases, so if the client is free for 1 hour he will consider 15, 30, 45 and 60 mins, then based on his experience, Reco will pick a time in each slot that is sufficient for each client. Note that, in this setting, you can start after the slot starting time, for example, if the client is free from 10:00 AM - 11:00 AM and he needs 45 mins, Reco will consider two options, either 10:00 AM - 10:45 AM or 10:15 AM-11:00 AM.


<img src="../figures/weighted tasks.PNG" width="600px"/>

Reco recognized that the problem can be modeled as an acyclic directed graph(DAG), and the solution of the problem is the longest path in that graph.

<img src="../figures/Constraint graph.PNG" width="600px"/>

In general, the longest paths are hard problems, actually, they are NP-Complete. However, fortunately, the longest path in DAG is easy, they could be solved by dynamic programming (DP) algorithm.


In [2]:
from collections import defaultdict
tasks = [("T1", 8.30, 9.30, 0.30, 3), 
         ("T2", 9.15, 10.15, 0.45, 4),
         ("T3", 9.30, 10.45, 1.15, 2),
         ("T4", 10.15, 11.30, 0.15, 1),
         ("T5", 12.00, 13.30, 0.30, 5),
         ("T6", 12.15, 13.45, 0.45, 2)]

tasks = sorted(tasks, key=lambda x: x[2])

def base_60_to_base_10(t):
    t_hr, t_min = str(t).split(".")
    t_hr = float(t_hr)
    t_min = float(t_min)
    if t_min < 10:
        t_min = t_min * 10
    t = t_hr + (t_min/60.0)
    return t

def base_10_to_base_60(t):
    t_hr, t_min = str(t).split(".")
    t_hr = float(t_hr)
    t_min = float(t_min)
    if t_min < 10:
        t_min = t_min * 10
    t = t_hr + (t_min/100.0 * 0.6)
    return t


def find_possible_slots(start_time, end_time, duration):
    for i in range(int((end_time - start_time) / 0.25)):
        end_time_ = start_time + i * 0.25 + duration
        if end_time_ <= end_time:
            yield (start_time + i * 0.25, start_time + i * 0.25 + duration, duration)
            
            
def connect_tasks_to_successor(tasks, i):
    #tasks that will start after i finish and start <= finish of connected to task
    #we dont connect last task
    if i >= len(tasks) - 1:
        return
    
    to_connect = []
    task = tasks[i]
    max_successor_start = task["finish"]
    for task_ in tasks[i+1:]:
        if task["task_group"] == task_["task_group"]:
            continue
        if task_["start"] >= task["finish"] and (task_["start"] <= max_successor_start or len(to_connect) == 0):
            to_connect += [task_["task"]]
            max_successor_start = max(task_["finish"], max_successor_start)
            
    return to_connect


tasks_ = []
for task in tasks:
    slots = list(find_possible_slots(base_60_to_base_10(task[1]), base_60_to_base_10(task[2]), base_60_to_base_10(task[3])))
    for slot in slots:
        tasks_ += [{"task": task[0] + "_" + str(slot[0]) + "_" + str(slot[1]), "task_group": task[0], "start": slot[0], "finish": slot[1], "duration": slot[2], "weight": task[4]}]
tasks_  = sorted(tasks_, key=lambda x: x["finish"])

#build the DAG
DAG = []
for i in range(0, len(tasks_)):
    task = {"task": tasks_[i]["task"], "info": tasks_[i], "next_tasks": connect_tasks_to_successor(tasks_, i)}
    DAG += [task]
    
best_weight = defaultdict(float)
best_from = defaultdict(str)
weight = {task["task"]: task["info"]["duration"]  for task in DAG}

for i in DAG:
    if i["next_tasks"] is not None:
        for next_task in i["next_tasks"]:
            if weight[next_task] + best_weight[i["task"]] > best_weight[next_task]:
                best_weight[next_task] = weight[next_task] + best_weight[i["task"]]
                best_from[next_task] = i["task"]
                
last_task = sorted(best_weight.items(), key=lambda i:i[1])[-1][0]
path = [last_task]
while best_from[last_task] != "":
    path += [best_from[last_task]]
    last_task = best_from[last_task]
    
path


['T6_13.0_13.75',
 'T5_12.0_12.5',
 'T4_10.75_11.0',
 'T3_9.5_10.75',
 'T1_8.5_9.0']

### Version 3: let's face it, no algorithm is enough!

Great! reko is almost there, the DP algorithm collects x units of utility. Reco have one last riddle to solve, in reality, each call has a probabilistic output. For example, if he spends 45 Mins with the client 1 there is 85% probability to succeed and 15% to fail and gain nothing, in the other hand if he spends only 15 Mins the client will not feel comfortable and the probability reduces to 35% of success and 65% of failure. This introduces two problems, first: what does it mean to find an optimal solution when the output is probabilistic. And second, even if we could solve it, from where on plant earth we could get these probabilities! Reco took a large cup of coffee and went on deep thinking, after a couple of hours Reco recognized that the algorithm should mutually optimize for total value and learn the probabilities on the way. Does the combo of optimization and learning sound familiar to you? of course, this is what bandits do.


> **_NOTE:_** it is common that constraints of real-world problems make the algorithm design a hard task. Instead, model the problem and let the solvers do the heavy lifting for you ;)

* Issue 1: How to overcome the pathological case of our algorithm
* Issue 2: How to account for the probabilistic outcome 
* Issue 3: How to solve the problem with missing info

the dynamic programming worked well for this example, however, there are few problems with it: first, for cases of small meeting time with longer slot available, the algorithm may pic the same slot multiple times (as in example below), and second what if the client is providing multiple available slots in his calendar, the same client may be picked for multiple meetings in one day. Although you can modify your algorithm to avoid these cases it is most likely not a good idea. In the real world scenarios, the problem will be modified in various ways that make the algorithm design a hard task and you will need to revamp your algorithm over and over.

For example, think about what will happen if you are asked to use the algorithm with a team of marketing agents instead of one, and what if each of them has different availability time and workload? these modifications will make the problem harder and most likely NP-Complete and there will not be a good algorithm that scale to more than a few marketing slots per calendar. In that case, to avoid being fired, you want to design an algorithm that results in good enough solutions instead of an optimal one.  

Instead of working hard on the algorithm, we want to work hard on modeling the problem and use a tool that comes up with good solutions, even in case the problem is NP-Complete. This will allow us to save the algorithm development time and make iterative solutions by changing the objective and adding constraints to our problem. Fortunately, for a large portion of such a problem, there are optimization methods that could be used.


In [ ]:
# !pip install pyomo
# !pyomo install-extras
#!conda install -y -c conda-forge pyomo.extras
#!conda install -y -c conda-forge glpk
import random
import numpy as np
import pandas as pd
import pdb
import math
%matplotlib inline 

#### Knapask problem
To best demonistrate the idea lets take a simple example before going further. Suppose you are planning a trip and want to take some items with you, however, your car has limited capacity and you want to pick the items that are most valuable for you. suppose you have a hammer, wrench, screwdriver, towel. The weights and importance of these items are listed in table x. This problem is known as the knapsack problem and it is known to be an NP-Complete problem.

> **_NOTE:_** NP-Complete problem does not mean problem is unsolvable. TODO clarify!

To solve the problem we will use pyomo which is a great python package that wraps multiple optimizers and provide an elegant way of modeling optimization problems.

First thing we define the data model, it includes the item, the importance, and the weight of each item. To model the problem we should define a few things:
- **Optimization variable**, in this case, a boolean variable per items that indicate if it should be selected for the trip or not.
- **Objective we are trying to optimize**, in this case, the sum of values of items that we can carry
- **Constraints** that are imposed on our problem, here the sum of picked items wait are less than 14

Fortunately, these constructs are available in pyomo and are intuitive to use. You can just define variables, objectives, and constraints and attach them to a model. After that, you just initiate a solver and voila, you have your answer!



In [8]:
from pyomo.environ import *
import pyomo.environ as pe 

# the data
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

#the model 
model = ConcreteModel()

# the variables
model.x = Var( A, within=Binary )

# the objective 
model.value = Objective(expr = sum( b[i]*model.x[i] for i in A), sense = maximize )

# weights 
model.weight = Constraint(expr = sum( w[i]*model.x[i] for i in A) <= W_max )

#solver 
opt = SolverFactory('glpk')

#Voela !
result_obj = opt.solve(model) 
model.x.get_values()

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

#### Modifying the problem
now let's say you decided that either hammer or wrench should be picked. No problems, you go the model, define a constraint and run again and that's it. You can see now that instead of focusing on the algorithm itself, you have room to focus on the problem and let pyomo do the heavy lifting.

In [10]:
model.c = Constraint(expr = sum(model.x[i] for i in [A[1], A[3]]) == 1)

#### A sanity check baseline
So how to know that the solution is good enough after all. A good idea is to build some baseline to compare with because we don't care about the perfect solution the baseline should be simple enough. in this example, let's say we pick the highest value fist until we don't have space.

In [9]:
#code

the solution of the baseline is wrench, hammer, and towel which sum up to 14 the same as ... (TODO find better weights)

#### Scheduling as an optimization problem
Now, let's return to our scheduling problem. We define the optimization problem as follows:

- **Variables**: boolean variable for each possible slot to be picked or not.
- **Objective**: maximize the sum of value from scheduled meetings
- **Constraint**: sum(slots per client) <= 1 for each client (each client could be contacted at most once per day)
- **Constraint**: Sum(slots) <= 1 for each overlap

and pretty much that's it, now lets put it into code:

**1. Generate sample problems**: instead of using fixed problem we generate sample problems for testing our algorithms. for each client, we generate 1-2 available times per day, each of these could randomly start between 8 and 14 and have a random duration of 15, 30, 45 or 60 mins. And finally, each available time is associated with some random weight.


In [1]:
def generate_schedual(clients):
    schedual = []
    for client in clients:
        num_slots = random.randint(1, 2)
        for i in range(num_slots):
            start = float(random.randint(8, 14)) + random.randint(0, 4) * 0.25
            duration = 0.25 * random.randint(1, 4)
            end = start + duration
            weight = random.randint(1, 5)
            schedual += [[client, start, end, duration, weight]]
    return schedual

**2. Enumerate slots** each available time per client may have different solts, we enumerate them all and store them into a pandas data frame for convenience.

In [50]:
from collections import defaultdict
# tasks = [["T1", 08.50, 09.50, 1.00, 3], 
#          ["T2", 09.25, 10.00, 0.75, 4],
#          ["T3", 09.50, 10.75, 0.50, 2],
#          ["T4", 10.25, 11.50, 0.25, 1],
#          ["T5", 12.00, 13.50, 0.75, 1],
#          ["T6", 12.25, 13.75, 0.75, 2],
#          ["T1", 12.00, 13.50, 1.00, 3]]

tasks = generate_schedual(["T1", "T2", "T3", "T4", "T5", "T6"])
def find_possible_slots(start_time, end_time, duration):
    for i in range(int((end_time - start_time) / 0.25)):
        end_time_ = start_time + i * 0.25 + duration
        if end_time_ <= end_time:
            yield (start_time + i * 0.25, start_time + i * 0.25 + duration, duration)

def task_id(task, slot):
    return {"task": task[0] + "_" + str(slot[0]) + "_" + str(slot[1]), 
            "task_group": task[0], 
            "start": slot[0], 
            "finish": slot[1], 
            "duration": slot[2], 
            "weight": task[4]}

def compute_possible_tasks(tasks):
    tasks_ = []
    for task in tasks:
        slots = find_possible_slots(float(task[1]), float(task[2]), float(task[3]))
        for slot in slots:
            tasks_ += [task_id(task, slot)]
    tasks_  = sorted(tasks_, key=lambda x: x["finish"])
    return pd.DataFrame(tasks_)

**3. Find overlaps** We find overlapped tasks using pandas, please not this implementation is not efficient however we use it here for illustration purposes, in production you need to consider using better data structure like segment tree to find overlapping tasks efficiently.

In [51]:
def find_task_overlaps(possible_tasks):
    overlapping_tasks = {}
    for task in possible_tasks.task.unique():
        start, finish = possible_tasks[possible_tasks.task == task].iloc[0][["start", "finish"]].values
        overlapping_tasks_ = possible_tasks[(possible_tasks.start >= start) & (possible_tasks.start < finish)]
        overlapping_tasks_ = list(overlapping_tasks_.task.unique())
        if len(overlapping_tasks_) > 1:
            overlapping_tasks[task] = overlapping_tasks_
    return overlapping_tasks


**4. Optimize** we define the optimization problem. (break it down)

In [52]:
def schedual_tasks(tasks):
    model = ConcreteModel()

    #data
    possible_tasks = compute_possible_tasks(tasks)
    tasks = possible_tasks.task.values
    w = {r.task: r.weight for _, r in possible_tasks[possible_tasks.task == tasks].iterrows()}

    #constarint data
    overlapping_tasks = find_task_overlaps(possible_tasks)
    task_groups = possible_tasks["task_group"].unique()

    #variables
    model.x = Var(tasks, within=Binary )

    #objective
    model.value = Objective(expr = sum(w[i]*model.x[i] for i in tasks), sense = maximize )

    #constraints
    @model.Constraint(task_groups)
    def one_each_group(m, tg):
        return sum(m.x[task] for task in possible_tasks[possible_tasks["task_group"] == tg]["task"].unique()) <= 1

    @model.Constraint(overlapping_tasks.keys())
    def one_each_overlap(m, t):
        return sum(m.x[task] for task in overlapping_tasks[t]) <= 1

    #solve
    opt = SolverFactory('glpk')
    result_obj = opt.solve(model)
    selected = [k for k, v in model.x.get_values().items() if v == 1]
    
    #formate resutls
    results = (possible_tasks
          .loc[possible_tasks.task.isin(selected)]
          .sort_values(by=['start'])
          .set_index("task_group")
          [["start", "finish", "duration", "weight"]])
    return results


In [53]:
%%capture
results = schedual_tasks(tasks)


In [54]:
results

,start,finish,duration,weight
task_group,,,,
T6,10.00,10.25,0.25,5
T4,10.25,10.50,0.25,1
T1,11.75,12.00,0.25,3
T3,12.75,13.25,0.50,5
T2,13.25,14.00,0.75,3
T5,14.25,14.75,0.50,3


In [55]:
results.weight.sum()

20

**5. Baseline sanity check** to verify that our solution is working fine, we set a greedy algorithm that takes the most valuable tasks first and eliminates all conflicting tasks. 

In [11]:
def solve_by_elemination(tasks):
    schedual = []
    possible_tasks = compute_possible_tasks(tasks)
    possible_tasks_ = possible_tasks.copy().sort_values(by=['weight'], ascending=False)
    for i in range(100):
        try:
            task, task_group, start, finish = possible_tasks_.iloc[i][["task", "task_group", "start", "finish"]]
            possible_tasks_ = possible_tasks_[~((possible_tasks_.start >= start) 
                                                & (possible_tasks_.start < finish) 
                                                & ((possible_tasks_.task != task)))]
            possible_tasks_ = possible_tasks_[~((possible_tasks_.finish > start) 
                                                & (possible_tasks_.finish <= finish) 
                                                & ((possible_tasks_.task != task)))]
            possible_tasks_ = possible_tasks_[(possible_tasks_.task_group != task_group) 
                                              | (possible_tasks_.task == task ) ]
            schedual += [task]
        except:
            break
    return possible_tasks[possible_tasks.task.isin(schedual)]

solve_by_elemination(tasks).weight.sum()

NameError: name 'compute_possible_tasks' is not defined

In this case, we see that our optimization is surpassing the baseline and the results make sense.

### Version 4: We don't know the weights

Looking back to what we have right now, we have clients that we want to meet during a day, each client provide his free time slots, and each meeting have specific duration and importance associated with it, your job is to schedule the meetings to maximize the importance you get from the meetings. You have realized the problem complexity, switch gears from building a complex algorithm to build a simple model and use pyomo optimization library to solve it, so far so good.

Now, you can imagine that the importance of each client is not pre defined, actually, we learned it on the way. Is it possible to extend our algorithm to optimize the schedule and learn the importance at the same time?. To answer this question, let's focus on the learning aspect. Imagine that you are traveling to a new country, you want to get some lunch and you find 2 restaurants nearby and you want to know which one to choose. Your measure from quality is on a scale from 0 to 1, where 0 means you don't like the food at all and 1 for perfect food. If you tried one restaurant for 10 times, you can compute the mean and standard deviation and form a confidence interval of the food quality at that restaurant. Comparing two restaurants is somehow tricky. If one restaurant got a 70% score and another get 75%, you could expect that the second is a better one, however it could be due to random reason. To establish a safe decision, you could use statistical procedures like T-Test or ANOVA to test if the difference is statistically significant or not.  


In [ ]:
#T-Test/ANOVA

While with the T-test it is pretty much safe to decide which restaurant is the best choice, you will need to try them all n times to draw the conclusion. That is simply not how we will solve it as humans, we don't try bad restaurants a hundred times to establish statistical significance! Instead, people will try restaurants at random and come back to the best ones they found so far, sometimes they will explore new options. This key in this method is to balance the exploration of new restaurants and the exploitation of the best restaurants. The value you loose due to exploring none optimal choices is called regret, and you basically want to minimize your regret of going to bad restaurants. Fortunately, the exploration vs. exploitation problem known as Multi-Arm Bandit MAB (TODO: add info in a side box) is well studied one, and in fact, there are provably optimal strategies to solve it. In general, we could rely on the principle of optimism in the face of uncertainty (TODO: add infobox), the idea is to choose the best know choices when we are not certain.

TODO: explain the UCB1 algorithm 


In [ ]:
#UCB1 and regret plot

Going from restaurants to optimization algorithms, we want to utilize the same principle optimism in the face of uncertainty. To make it possible it is better to model the optimization algorithm like an agent that interacts with some environment to learn about the problem and find an optimal solution. Our model has the following components:

- **Problem**: This encapsulates the objective, the constraints, the structure of the problem and all other info except for the problem wights.
- **Parameters**: encapsulate the weight of the problem 
- **Solver**: the optimizer we use to solve the problem 
- **Oracle**: the environment sensor that will tell us the weights of the solution
- **Agent**: This takes the problem, parameters, and a solver to solve the problem and then use the oracle to observe the solution quality.

#TODO draw a schematic of the env, infobox about AI agent architecture


In [3]:
class Oracle:
    def observe(self, problem, solution):
        pass
        
class Solver:
    def solve(self, problem, weights):
        pass
    
class Problem:
    pass

class ProblemModel:
    def get_all_weights(self):
        pass
    
    def set_weights(self, weights):
        pass
    


For our scheduling example, the problem will hold the meetings informations

In [6]:
class TaskSchedualingProblem(Problem):
    def __init__(self, tasks):
        self.tasks = tasks


The parameters will store the weights of each possible meeting, note that all possible tasks belong to the parent task have the same weight. For convenience, we set two modes to initialize the weights random and know waits. Know weights will be used to solve the problem as we know it, while random initial weights will be used whenever are in bandit mode. (TODO: rephrase)

In [7]:
class TaskSchedualingProblemModel(ProblemModel):
    def __init__(self, tasks, weights_known=False):
        if weights_known:
            self.weights = dict(compute_possible_tasks(tasks)[["task_group", "weight"]].drop_duplicates().values)
        else:
            self.weights = {i: random.random() for i in compute_possible_tasks(tasks)["task_group"].drop_duplicates().values}
        self.arms = self.weights.keys()  
        
    def get_all_weights(self):
        return self.weights
    
    def set_weights(self, weights):
        for k, v in weights.items():
            self.weights[k] = v
    



The solver will simply call the schedule tasks function we defined before


In [9]:
class TaskSchedualingSolver(Solver):
    def __init__(self):
        pass 
    
    def solve(self, problem, weights):
        tasks = [task[:4] + [weights[task[0]]] for task in problem.tasks]
        return schedual_tasks(tasks).index.values

The Oracle models our environment, depending on the nature of the environment the observations could be true values or noisy readings. 

In [10]:
class TaskSchedualingOracle(Oracle):
    def __init__(self, tasks, noise_factor=3.0):
        self.weights = dict(compute_possible_tasks(tasks)[["task_group", "weight"]].drop_duplicates().values)
        self.arms = self.weights.keys()  
        self.noise_factor = noise_factor
        
    def get_weight(self, x, noisy=False):
        if noisy:
            return self.weights[x] + (random.random() - 0.5) * self.noise_factor
        else:
            return self.weights[x]
        
    def observe(self, problem, solution, noisy=False):
        return [self.get_weight(x, noisy=noisy) for x in solution]


Something

In [1545]:
problem = TaskSchedualingProblem(tasks)
problem_model = TaskSchedualingProblemModel(tasks, weights_known=True)
oracle = TaskSchedualingOracle(tasks, noise_factor=5)
solver = TaskSchedualingSolver()
sln = solver.solve(problem, problem_model.get_all_weights())
sum(oracle.observe(problem, sln, noisy=False))

12

Something

In [1546]:
problem_model_uncertain = TaskSchedualingProblemModel(tasks, weights_known=False)
sln = solver.solve(problem, problem_model_uncertain.get_all_weights())
sum(oracle.observe(problem, sln, noisy=False))


10

In [1]:
class CombUcb1:
    def __init__(self, problem, problem_model, solver, oracle, mode='Max'):
        self.problem = problem
        self.solver = solver
        self.oracle = oracle
        self.arms = problem_model.arms
        self.mode = mode
        self.init_algorithm()

    def init_algorithm(self):
        uu = {i: 0.0 for i in self.arms}
        w = {i: 0.0 for i in self.arms}
        t = 0
        for arm in uu.keys():
            if self.mode == 'Max':
                uu[arm] = 1.0
            elif self.mode == 'Min':
                uu[arm] = 0.0
            else:
                raise Exception('Mode is only Max or Min')
        solution_exists = True

        while ((self.mode == 'Min' and np.min(list(uu.values())) == 0)
                                        or (self.mode == 'Max' and np.max(list(uu.values())) == 1.0)):
            At = self.solver.solve(self.problem, uu)
            if At is None:
                break
            AtW = self.oracle.observe(self.problem, At, noisy=True)

            for idx, e in enumerate(At):
                w[e] = AtW[idx]
                if self.mode == 'Max':
                    uu[e] = 0.0
                elif self.mode == 'Min':
                    uu[e] = 1.0
                else:
                    raise Exception('Mode is only Max or Min')
            t += 1
        self.weights = w
        self.time_steps = {i: 1.0 for i in w.keys()}
        self.t = t

    def bandit_iter(self):
        weights = self.weights
        time_steps = self.time_steps
        t = self.t
        if self.mode == 'Max':
            u_ucb = {i: min(weights[i] + np.sqrt(1.5 * np.log(t)/time_steps[i]), 1.0) for i in weights.keys()}
        elif self.mode == 'Min':
            u_ucb = {i: max(weights[i] - np.sqrt(1.5 * np.log(t)/time_steps[i]), 0) for i in weights.keys()}
        else:
            raise Exception('Mode is only Max or Min')
        At = self.solver.solve(self.problem, u_ucb)
        wAt = self.oracle.observe(self.problem, At)
        for idx, e in enumerate(At):
            weights[e] = (time_steps[e] * weights[e] + wAt[idx]) / (time_steps[e] + 1)
            time_steps[e] += 1
        t += 1
        self.time_steps = time_steps
        self.weights = weights
        self.t = t

    def solve(self):
        weights = self.weights
        u_ucb = weights
        return self.solver.solve(self.problem, u_ucb)


In [1547]:
%%capture 
b = CombUcb1(problem=problem, problem_model=problem_model_uncertain, solver=solver, oracle=oracle, mode='Max')

for i in range(5):
    print (i)
    b.bandit_iter()

In [1548]:
sln = solver.solve(problem, b.weights)
sum(oracle.observe(problem, sln, noisy=False))

10

### Not only scheduling
wait, but we can also solve problems without knowing the parametric! What about other problems?


#### Knapsack with unknown weights 

#### Shortest path with unknown weights


### Extend to a team of phone marketers

In [1549]:
agents_workload = ("Alex", "Jennifer", "Andrew", "DeAnna", "Jesse")

clients = (
    "Trista", "Meredith", "Aaron", "Bob", "Jillian",
    "Ali", "Ashley", "Emily", "Desiree", "Byron")

In [1550]:
import numpy as np
import sklearn

def score(agent, client):
    try:
        s = 1 / (1 + math.exp(-np.dot(agents_v[agent], clients_v[client])))
    except:
        print (np.dot(agents_v[agent], clients_v[client]))
        return random.random()
    return s
    
def generate_matching_problem(agents, clients):
    num_samples = len(agents) + len(clients)
    samples = sklearn.datasets.make_swiss_roll(num_samples, noise=3, random_state=0)[0]
    random.shuffle(samples)
    clients_v = {clients[i]: samples[i] for i in range(len(clients))}
    agents_v = {agents[i]: samples[i] for i in range(len(agents))}
    match_scores = dict(
        ((agent, client), score(agent, client))
        for agent in agents_workloads
        for client in clients)

    client_time = {client: random.randint(1, 4) for client in clients}
    agents_workload = {agent: random.randint(2, 5) for agent in agents}
    
    return match_scores, client_time, agents_workload, clients_v, agents_v

In [1551]:
def solve_matching(match_scores, client_time, agents_workload):
    agents = agents_workloads.keys()
    clients = client_time.keys()
    
    model = pe.ConcreteModel()
    model.agents = agents_workloads.keys()
    model.clients = clients
    model.match_scores = match_scores
    model.agents_workload = agents_workload

    model.assignments = pe.Var(match_scores.keys(), domain=pe.Binary)
    model.objective = pe.Objective(
            expr=pe.summation(model.match_scores, model.assignments),
            sense=pe.maximize)

    @model.Constraint(model.agents)
    def respect_workload(model, agent):
        return sum(model.assignments[agent, client] * client_time[client] for client in model.clients) <= model.agents_workload[agent]

    @model.Constraint(model.clients)
    def one_agent_per_client(model, client):
        return sum(model.assignments[agent, client] for agent in model.agents) <= 1


    solver = pe.SolverFactory("glpk")
    solver.solve(model)
    sln = [k for k, v in model.assignments.get_values().items() if v == 1.0]
    return sln, sum(match_scores[i] for i in sln)
    

In [1552]:
def solve_matching_greedy(match_scores, client_time, agents_workload):
    agents = agents_workloads.keys()
    clients = client_time.keys()
    matching = sorted(match_scores.items(), key=lambda x: -x[1])
    clients_indicator = {client: 0 for client in clients}
    agents_workload_ = agents_workload.copy()
    sln = []
    for (agent, client), score in matching:
        if clients_indicator[client] == 0 and agents_workload_[agent] >= client_time[client]:
            clients_indicator[client] = 1
            agents_workload_[agent] -= client_time[client]
            sln += [(agent, client)]
    return sln, sum([match_scores[i] for i in sln])



In [1553]:
match_scores, client_time, agents_workload, clients_v, agents_v = generate_matching_problem(agents, clients)

In [1554]:
solve_matching(match_scores, client_time, agents_workload)

([('DeAnna', 'Meredith'),
  ('Andrew', 'Ashley'),
  ('Jesse', 'Jillian'),
  ('DeAnna', 'Emily'),
  ('Jesse', 'Byron'),
  ('Jennifer', 'Bob'),
  ('Alex', 'Trista')],
 7.0)

In [1555]:
solve_matching_greedy(match_scores, client_time, agents_workload)

([('Alex', 'Trista'),
  ('Jennifer', 'Bob'),
  ('Andrew', 'Meredith'),
  ('DeAnna', 'Aaron'),
  ('DeAnna', 'Jillian'),
  ('Jesse', 'Ashley')],
 6.0)

In [1556]:
# the matching optimization problem
class MatchingProblem(Problem):
    def __init__(self, match_scores, client_time, agents_workload, agents_v, clients_v):
        self.match_scores = match_scores
        self.agents_workload = agents_workload
        self.client_time = client_time
        self.features = {(agent, client): np.hstack([agents_v[agent], clients_v[client]]) 
                         for agent, client in match_scores.keys()}


class MatchingProblemModel(ProblemModel):
    def __init__(self, match_scores, weights_known=False):
        if weights_known:
            self.weights = match_scores.copy()
        else:
            self.weights = {i: random.random() for i in match_scores.keys()}
        self.arms = self.weights.keys()  
        
    def get_all_weights(self):
        return self.weights
    
    def set_weights(self, weights):
        for k, v in weights.items():
            self.weights[k] = v
    

class MatchingSolver(Solver):
    def __init__(self):
        pass 
    
    def solve(self, problem, weights):
        return solve_matching(weights, problem.client_time, problem.agents_workload)
        
        
class MatchingOracle(Oracle):
    def __init__(self, match_scores, noise_factor=3.0):
        self.weights = match_scores.copy()
        self.arms = self.weights.keys()  
        self.noise_factor = noise_factor
        
    def get_weight(self, x, noisy=False):
        if noisy:
            return self.weights[x] + (random.random() - 0.5) * self.noise_factor
        else:
            return self.weights[x]
        
    def observe(self, problem, solution, noisy=True):
        return [self.get_weight(x, noisy=noisy) for x in solution]
        
        

In [1590]:
# CombTS
class CombLinTs:
    def __init__(self, problem, p_lambda, p_sigma, solver, oracle):
        self.d = np.array(list(problem.features.values())).shape[1]
        self.p_lambda = p_lambda
        self.p_sigma = p_sigma
        self.sigma = (p_lambda ** 2) * np.eye(self.d)
        self.theta = np.zeros(self.d)
        self.solver = solver
        self.oracle = oracle
        
    def sample_theta(self):
        return np.random.multivariate_normal(self.theta, self.sigma)
    
    def update_params(self, wAt):
        theta = self.theta
        sigma = self.sigma
        
        n = len(wAt)
        
        for k, v in wAt.items():
            f_vec = np.expand_dims(problem.features[k], axis=1)
            t1 = np.matmul(sigma, np.matmul(f_vec, f_vec.T))
            t2 = np.matmul(f_vec.T, np.matmul(sigma, f_vec)) + self.p_sigma ** 2
            t3 = np.matmul(sigma, f_vec)
            t4 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            theta = np.matmul((np.eye(sigma.shape[0]) - t1 / t2), theta) + \
                        np.squeeze(t3 / t4) * wAt[k]

            t1 = np.matmul(np.matmul(sigma, np.matmul(f_vec, f_vec.T)), sigma)
            t2 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            sigma = sigma - t1/t2
    
        self.theta = theta
        self.sigma = sigma
            
    def bandit_iter(self, problem):
        theta = self.sample_theta()
        At, _ = self.solver.solve(problem, {k: np.dot(v, theta) for k, v in problem.features.items()})
        wAt = self.oracle.observe(problem, At, noisy=True)
        wAt = dict(zip(At, wAt))
        self.update_params(wAt)
        
    def solve(self, problem):
        theta = self.sample_theta()
        sln, _ = self.solver.solve(problem, {k: np.dot(v, theta) for k, v in problem.features.items()})
        return sln

In [1634]:
match_scores, client_time, agents_workload, clients_v, agents_v = generate_matching_problem(agents, clients)
problem = MatchingProblem(match_scores, client_time, agents_workload, agents_v, clients_v)
problem_model = MatchingProblemModel(match_scores, weights_known=False)
oracle = MatchingOracle(match_scores, noise_factor=2)
solver = MatchingSolver()
sln, w = solver.solve(problem, problem_model.get_all_weights())
np.sum(oracle.observe(problem, At, noisy=False))


6.00008340099822

In [1635]:
problem_model = MatchingProblemModel(match_scores, weights_known=True)
sln, w = solver.solve(problem, problem_model.get_all_weights())
np.sum(oracle.observe(problem, sln, noisy=False))


8.0

In [1638]:
p_lambda = 10.
p_sigma = 0.1
features_dim =  10
bandit = CombLinTs(p_lambda=p_lambda,
                     p_sigma=p_sigma, problem=problem, solver=solver, oracle=oracle)

for i in range(100):
    bandit.bandit_iter(problem=problem)
    if i % 10 == 0:
        At = bandit.solve(problem)
        print (np.sum(oracle.observe(problem, At, noisy=False)))

7.0
7.0
8.0
8.0
7.0
7.0
8.0
8.0
8.0
8.0


* scale up all pairs 
* We assumed all weight is random, what if they are normal with different means and variances
* Empirical Bayes for new joiners 
* Extend to team
* Spending more time is useful
* Streamline with Tensorflow probability

In [12]:
# TODO: the bandit example
# TODO: drwa gunt chart
# TODO: shortest path 
# TODO: Knapsack

You may want to do both selection and scheduling, learn the importance of each client, start from reasonable wights for faster exploration or add more cool features. Very well, we will go through after we introduce some cool tools. For the time being, let's sharpen our skills by solving 2 other problems.

**The dauntless conclusion**: *Be a modeling super star, dont fear missing info!*

### Bibliography (Text style like in introduction to algorithms)